In [ ]:
import streamlit as st

In [ ]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf
!pip -q install sentence_transformers
!pip install chromadb
!pip install faiss-cpu


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.3.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 

In [ ]:
from langchain.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from huggingface_hub import notebook_login
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
import os
import sys
import numpy as np
import faiss

In [ ]:
!mkdir docs

In [ ]:


document = []
for file in os.listdir("docs"):
    if file.endswith(".pdf"):
        pdf_path = "./docs/" + file
        loader = PyPDFLoader(pdf_path)
        document.extend(loader.load())
    elif file.endswith('.docx') or file.endswith('.doc'):
        doc_path = "./docs/" + file
        loader = Docx2txtLoader(doc_path)
        document.extend(loader.load())
    elif file.endswith('.txt'):
        text_path = "./docs/" + file
        loader = TextLoader(text_path)
        document.extend(loader.load())


In [ ]:
document_splitter = CharacterTextSplitter(separator='\n', chunk_size=500, chunk_overlap=100)
document_chunks = document_splitter.split_documents(document)


In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
document_texts = [doc.page_content for doc in document_chunks]
document_embeddings = embeddings.embed_documents(document_texts)
document_embeddings_array = np.array(document_embeddings)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Build the FAISS index
d = document_embeddings_array.shape[1]  # Dimension of the embeddings
index = faiss.IndexFlatL2(d)  # L2 distance (Euclidean)
index.add(document_embeddings_array)


In [ ]:
from langchain.vectorstores import Chroma
vectordb=Chroma.from_documents(document_chunks,embedding=embeddings, persist_directory='./data')

In [ ]:
vectordb

In [ ]:
vectordb.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [ ]:
notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large",
                                              device_map='auto',
                                              torch_dtype=torch.float16)

pipe = pipeline("text2text-generation",
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.float16,
                device_map='auto',
                max_length=512)

llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature': 0})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

retriever = vectordb.as_retriever(search_kwargs={'k': 6})
pdf_qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                               retriever=retriever,
                                               verbose=False,
                                               memory=memory)



In [ ]:
'''print('---------------------------------------------------------------------------------')
print('Welcome to the DocBot. You are now ready to start interacting with your documents')
print('---------------------------------------------------------------------------------')

while True:
  query=input(f"Prompt:")
  if query == "exit" or query == "quit" or query == "q" or query == "f":
    print('Exiting')
    sys.exit()
  if query == '':
    continue
  result = pdf_qa({"question": query})
  print(f"Answer: " + result["answer"])
'''

---------------------------------------------------------------------------------
Welcome to the DocBot. You are now ready to start interacting with your documents
---------------------------------------------------------------------------------
Answer: rangeerror


In [ ]:
print('---------------------------------------------------------------------------------')
print('Welcome to the DocBot. You are now ready to start interacting with your documents')
print('---------------------------------------------------------------------------------')
while True:
    query = input(f"Prompt:")
    if query.lower() in ["exit", "quit", "q", "f"]:
        print('Exiting')
        sys.exit()
    if query == '':
        continue
    # Perform FAISS similarity search
    query_embedding = np.array(embeddings.embed_documents([query])).reshape(1, -1)
    k = 6
    distances, indices = index.search(query_embedding, k)
    # Retrieve the documents and their scores
    results = [(document_chunks[idx], distances[0][i]) for i, idx in enumerate(indices[0])]
    for doc, score in results:
        print(f"Document: {doc.page_content}")
        print(f"Score: {score}")
    top_document = results[0][0].page_content if results else "No relevant documents found."
    result = pdf_qa({"question": query, "context": top_document})
    print(f"Answer: {result['answer']}")

---------------------------------------------------------------------------------
Welcome to the DocBot. You are now ready to start interacting with your documents
---------------------------------------------------------------------------------
Prompt:reachability analysis
Document: Reachability analysis (RA) has been widely employed to formally verify driving safety [5, 6]. RA computes
a complete set of states that an agent (e.g. a vehicle) can reach given an initial condition within a certain time
interval [7]. Based on RA, a safety verification thus can be performed by propagating all possible reachable
space of the AV and other traffic participants on the road. In doing so, safety is ensured if such forward
Score: 0.6569088697433472
Document: IEEE Transactions on Robotics , vol. 30, no. 4, pp. 903–918, 2014.
[7] M. Althoff, G. Frehse, and A. Girard, “Set propagation techniques for reachability analysis,” Annual
Review of Control, Robotics, and Autonomous Systems , vol. 4, pp. 369–

KeyboardInterrupt: 